# Pre-Processing

# Importing

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
from os import walk

path = "../data/raw/"
filenames = next(walk(path), (None, None, []))[2]
print(filenames)

['campaigns.csv', 'customers.csv', 'customer_reviews_complete.csv', 'interactions.csv', 'support_tickets.csv', 'transactions.csv']


In [3]:
campaigns = pd.read_csv(path+"campaigns.csv")
customers = pd.read_csv(path+"customers.csv")
customer_reviews_complete = pd.read_csv(path+"customer_reviews_complete.csv")
interactions = pd.read_csv(path+"interactions.csv")
support_tickets = pd.read_csv(path+"support_tickets.csv")
transactions = pd.read_csv(path+"transactions.csv")

# Cleaning

## Missing Values

### Discover

In [4]:
print(campaigns.isnull().all().sum())
print(customers.isnull().all().sum())
print(customer_reviews_complete.isnull().all().sum())
print(interactions.isnull().all().sum())
print(support_tickets.isnull().all().sum())
print(transactions.isnull().all().sum())

0
0
0
0
0
0


In [5]:
pd.DataFrame(campaigns.isna().sum()).T / campaigns.shape[0]

,campaign_id,campaign_name,campaign_type,start_date,end_date,target_segment,budget,impressions,clicks,conversions,conversion_rate,roi
0,0.0,0.03,0.03,0.0,0.0,0.0,0.02,0.02,0.05,0.035,0.01,0.015


In [6]:
pd.DataFrame(customers.isna().sum()).T / customers.shape[0]

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,0.0,0.0212,0.0372,0.0224,0.0222,0.0372,0.0354,0.0194,0.0186,0.0192,0.0,0.0228


In [7]:
pd.DataFrame(customer_reviews_complete.isna().sum()).T / customer_reviews_complete.shape[0]

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
0,0.0,0.0,0.024,0.019,0.019,0.0,0.0,0.0,0.0,0.0


In [8]:
pd.DataFrame(interactions.isna().sum()).T / interactions.shape[0]

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
0,0.0,0.0,0.02002,0.02022,0.0,0.01963,0.01927,0.0


In [9]:
pd.DataFrame(support_tickets.isna().sum()).T / support_tickets.shape[0]

,ticket_id,customer_id,issue_category,priority,submission_date,resolution_date,resolution_status,resolution_time_hours,customer_satisfaction_score,notes
0,0.0,0.0,0.023667,0.02,0.0,0.081333,0.019,0.099667,0.113667,0.020333


In [10]:
pd.DataFrame(transactions.isna().sum()).T / transactions.shape[0]

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
0,0.0,0.0,0.020994,0.021242,0.019941,0.01926,0.0,0.019941,0.020437,0.018919


### Rectify

Using mean values to interpolate numeric values.<br>
Using "Other" for missing text values.

#### 1. campaigns

In [11]:
def fill_avg (df, na_col, ref_col):
    items = list(df[df[na_col].isna()][ref_col])
    for item in items:
        mean = df[df[ref_col] == item][na_col].mean()
        df.loc[(df[ref_col] == item) & (df[na_col].isna()), na_col] = mean
    return

In [12]:
campaigns = campaigns.fillna(value={"campaign_name": "Other", "campaign_type": "Other"})

In [13]:
campaigns.loc[(campaigns["campaign_type"] == "Email Marketing") & (campaigns["budget"].isna()), "budget"]

35   NaN
Name: budget, dtype: float64

In [14]:
def campaign_conversion_rate (row):
    if pd.isna(row["conversion_rate"]):
        return row["conversions"] / row["clicks"]
    else:
        return row["conversion_rate"]
        

In [15]:
fill_avg(campaigns, "budget", "campaign_type")
fill_avg(campaigns, "impressions", "campaign_type")
fill_avg(campaigns, "clicks", "campaign_type")
fill_avg(campaigns, "conversions", "campaign_type")
campaigns["conversion_rate"] = campaigns.apply(campaign_conversion_rate, axis=1)
fill_avg(campaigns, "roi", "campaign_type")

In [16]:
pd.DataFrame(campaigns.isna().sum()).T / campaigns.shape[0]

,campaign_id,campaign_name,campaign_type,start_date,end_date,target_segment,budget,impressions,clicks,conversions,conversion_rate,roi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 2. customers

In [17]:
customers = customers.fillna(value={"full_name": "John Doe", 
                                    "gender": "Prefer not to say", 
                                    "email": "none@none.none", 
                                    "phone": "127-0-0-1", 
                                    "street_address": "Other", 
                                    "city": "Other", 
                                    "state": "Other", 
                                    "zip_code": 99999, 
                                    "preferred_channel": "Other"})
customers["age"] = customers["age"].apply(lambda age: customers["age"].mean() if pd.isna(age) else age)

In [18]:
pd.DataFrame(customers.isna().sum()).T / customers.shape[0]

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 3. customer_reviews_complete

In [19]:
def customer_reviews_complete_full_name (row):
    if pd.isna(row["full_name"]):
        customer_id = row["customer_id"]
        full_name = customers[customers["customer_id"] == customer_id]["full_name"]
        return full_name
    else:
        return row["full_name"]

In [20]:
customer_reviews_complete = customer_reviews_complete.fillna(value={"product_name": "Other", "product_category": "Other"})
customer_reviews_complete["full_name"] = customer_reviews_complete.apply(customer_reviews_complete_full_name, axis=1)

In [21]:
pd.DataFrame(customer_reviews_complete.isna().sum()).T / customer_reviews_complete.shape[0]

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 4. interactions

In [22]:
interactions = interactions.fillna(value={"channel": "Other", "interaction_type": "Other", "page_or_product": "Other"})
interactions["duration"] = interactions["duration"].apply(lambda dur: interactions["duration"].mean() if pd.isna(dur) else dur)

In [23]:
pd.DataFrame(interactions.isna().sum()).T / interactions.shape[0]

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 5. support_tickets

In [24]:
def support_tickets_resolution_status (row):
    if pd.isna(row["resolution_status"]):
        if pd.isna(row["resolution_date"]):
            return "Pending"
        else:
            return "Closed without Resolution"
    else:
        return row["resolution_status"]

In [25]:
def support_tickets_resolution_time_hours (row):
    if pd.isna(row["resolution_time_hours"]):
        return np.floor((row["resolution_date"] - row["submission_date"]).seconds / 3600)
    else:
        return row["resolution_time_hours"]

In [26]:
support_tickets = support_tickets.fillna(value={"issue_category": "Other", 
                                                "priority": "Medium", 
                                                "notes": "No Description", 
                                                "customer_satisfaction_score":3.0 })
support_tickets["resolution_status"] = support_tickets.apply(support_tickets_resolution_status, axis=1)
support_tickets["submission_date"] = pd.to_datetime(support_tickets["submission_date"])
support_tickets["resolution_date"] = pd.to_datetime(support_tickets["resolution_date"])
support_tickets["resolution_date"] = support_tickets["resolution_date"].apply(lambda dat: support_tickets["resolution_date"].max() if pd.isna(dat) else dat)
support_tickets["resolution_time_hours"] = support_tickets.apply(support_tickets_resolution_time_hours, axis=1)

In [27]:
pd.DataFrame(support_tickets.isna().sum()).T / support_tickets.shape[0]

,ticket_id,customer_id,issue_category,priority,submission_date,resolution_date,resolution_status,resolution_time_hours,customer_satisfaction_score,notes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 6. transactions

In [28]:
transactions = transactions.fillna(value={"product_name": "Other", "product_category": "Other", "store_location": "Online", "payment_method": "Credit Card"})
transactions["quantity"] = transactions["quantity"].apply(lambda quant: 1 if pd.isna(quant) else quant)
fill_avg(transactions, "price", "product_name")
transactions["discount_applied"] = transactions["discount_applied"].apply(lambda disc: 0.0 if pd.isna(disc) else disc)

In [29]:
pd.DataFrame(transactions.isna().sum()).T / transactions.shape[0]

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data Types

### Discover

In [30]:
campaigns = campaigns.convert_dtypes()
customers = customers.convert_dtypes()
customer_reviews_complete = customer_reviews_complete.convert_dtypes()
interactions = interactions.convert_dtypes()
support_tickets = support_tickets.convert_dtypes()
transactions = transactions.convert_dtypes()

#### 1. campaigns

In [31]:
campaigns.head(1)

,campaign_id,campaign_name,campaign_type,start_date,end_date,target_segment,budget,impressions,clicks,conversions,conversion_rate,roi
0,0d120125-26cc-4023-9a54-bebbc0159b76,Black Friday Sale 2022,Online Display Ads,2020-09-29,2020-11-13,Southern States,14528.9,740965.0,1189.0,53.0,4.46,-61.31


In [32]:
pd.DataFrame(campaigns.dtypes).T

,campaign_id,campaign_name,campaign_type,start_date,end_date,target_segment,budget,impressions,clicks,conversions,conversion_rate,roi
0,string[python],string[python],string[python],string[python],string[python],string[python],Float64,Float64,Float64,Float64,Float64,Float64


#### 2. customers

In [33]:
customers.head(1)

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,4c30e132-0704-4459-a509-9eddde934977,Mark Johnson,40.0,Male,mark.johnson@yahoo.com,989.608.3863,819 Johnson Course,Houston,Texas,29158,2024-04-25,Other


In [34]:
pd.DataFrame(customers.dtypes).T

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,string[python],string[python],Float64,string[python],string[python],string[python],string[python],string[python],string[python],Int64,string[python],string[python]


#### 3. customer_reviews_complete

In [35]:
customer_reviews_complete.head(1)

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
0,rev_000001,e9848049-323f-4f95-a26e-3cffc219d591,Bookshelf,Furniture,Patrick Guerrero,2024-05-30,2024-06-12,5,Sturdy and Elegant Addition to My Study Room,Just received the bookshelf I've been eyeing f...


In [36]:
pd.DataFrame(customer_reviews_complete.dtypes).T

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
0,string[python],string[python],string[python],string[python],object,string[python],string[python],Int64,string[python],string[python]


#### 4. interactions

In [37]:
interactions.head(1)

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
0,423db3d3-659d-433f-859a-eea2ff1e2d2c,00012aa8-e99c-4e30-b3f6-1f7e36adc517,Other,review,2023-03-11 08:37:00,128.0,home,00012aa8-e99c-4e30-b3f6-1f7e36adc517_session_0


In [38]:
pd.DataFrame(interactions.dtypes).T

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
0,string[python],string[python],string[python],string[python],string[python],Float64,string[python],string[python]


#### 5. support_tickets

In [39]:
support_tickets.head(1)

,ticket_id,customer_id,issue_category,priority,submission_date,resolution_date,resolution_status,resolution_time_hours,customer_satisfaction_score,notes
0,b38add31-c7bf-4dcd-ae90-6d7c3cca0728,20b8b390-06f6-4558-9641-9d87ca9de14c,Technical,Medium,2024-10-17,2024-10-18 11:00:00,Resolved,35,5,Customer experiencing software issues with Cof...


In [40]:
pd.DataFrame(support_tickets.dtypes).T

,ticket_id,customer_id,issue_category,priority,submission_date,resolution_date,resolution_status,resolution_time_hours,customer_satisfaction_score,notes
0,string[python],string[python],string[python],string[python],datetime64[ns],datetime64[ns],string[python],Int64,Int64,string[python]


#### 6. transactions

In [41]:
transactions.head(1)

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
0,53e3d9f5-6231-46de-aa39-e5f8eaae2eee,727839b2-f084-4e94-94d8-ae59cc8e4b84,Ring Doorbell,Smart Home Devices,1,140.07,2020-12-06,"Houston, TX",Credit Card,20


In [42]:
pd.DataFrame(transactions.dtypes).T

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
0,string[python],string[python],string[python],string[python],Int64,Float64,string[python],string[python],string[python],Int64


### Rectify

#### 1. campaigns

In [43]:
campaigns["start_date"] = pd.to_datetime(campaigns["start_date"])
campaigns["end_date"] = pd.to_datetime(campaigns["end_date"])

In [44]:
pd.DataFrame(campaigns.dtypes).T

,campaign_id,campaign_name,campaign_type,start_date,end_date,target_segment,budget,impressions,clicks,conversions,conversion_rate,roi
0,string[python],string[python],string[python],datetime64[ns],datetime64[ns],string[python],Float64,Float64,Float64,Float64,Float64,Float64


#### 2. customers

In [45]:
customers["registration_date"] = pd.to_datetime(customers["registration_date"])

In [46]:
pd.DataFrame(customers.dtypes).T

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,string[python],string[python],Float64,string[python],string[python],string[python],string[python],string[python],string[python],Int64,datetime64[ns],string[python]


#### 3. customer_reviews_complete

In [47]:
customer_reviews_complete["full_name"] = customer_reviews_complete["full_name"].astype("string")
customer_reviews_complete["transaction_date"] = pd.to_datetime(customer_reviews_complete["transaction_date"])
customer_reviews_complete["review_date"] = pd.to_datetime(customer_reviews_complete["review_date"])

In [48]:
pd.DataFrame(customer_reviews_complete.dtypes).T

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
0,string[python],string[python],string[python],string[python],string[python],datetime64[ns],datetime64[ns],Int64,string[python],string[python]


#### 4. interactions

In [49]:
interactions["interaction_date"] = pd.to_datetime(interactions["interaction_date"])

In [50]:
pd.DataFrame(interactions.dtypes).T

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
0,string[python],string[python],string[python],string[python],datetime64[ns],Float64,string[python],string[python]


#### 5. support_tickets

In [51]:
support_tickets["submission_date"] = pd.to_datetime(support_tickets["submission_date"])
support_tickets["resolution_date"] = pd.to_datetime(support_tickets["resolution_date"])

In [52]:
pd.DataFrame(support_tickets.dtypes).T

,ticket_id,customer_id,issue_category,priority,submission_date,resolution_date,resolution_status,resolution_time_hours,customer_satisfaction_score,notes
0,string[python],string[python],string[python],string[python],datetime64[ns],datetime64[ns],string[python],Int64,Int64,string[python]


#### 6. transactions

In [53]:
transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"])

In [54]:
pd.DataFrame(transactions.dtypes).T

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
0,string[python],string[python],string[python],string[python],Int64,Float64,datetime64[ns],string[python],string[python],Int64


## Faulty Values

### Discover

In [55]:
def unique_perc (df):
    uniques = {}
    for column in df.columns:
        uniques[column] = [df[column].unique().shape[0] / df[column].shape[0]]
    return pd.DataFrame(uniques)

def unique_vals (df, columns, filename=None):
    path_log = "../log/"
    uniques = {}
    for column in columns:
        uniques[column] = list(df[column].unique())
    if filename:
        with open(path_log+filename+".json", "w", encoding="utf-8") as file:
            json.dump(uniques, file, indent=4)
    return

#### 1. campaigns

In [56]:
unique_perc(campaigns)

,campaign_id,campaign_name,campaign_type,start_date,end_date,target_segment,budget,impressions,clicks,conversions,conversion_rate,roi
0,1.0,0.875,0.055,0.94,0.915,0.09,1.0,0.995,0.975,0.935,0.88,1.0


In [57]:
unique_vals(campaigns, ["campaign_type", "target_segment"], "campaigns")

#### 2. customers

In [58]:
unique_perc(customers)

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
0,1.0,0.9442,0.0116,0.0008,0.9554,0.963,0.9648,0.015,0.0032,0.958,0.3422,0.0008


In [59]:
unique_vals(customers, ["age", "gender", "city", "state", "preferred_channel"], "customers")

#### 3. customer_reviews_complete

In [60]:
unique_perc(customer_reviews_complete)

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
0,1.0,0.83,0.075,0.016,0.826,0.65,0.673,0.005,0.957,1.0


In [61]:
unique_vals(customer_reviews_complete, ["product_name", "product_category"], "customer_reviews_complete")

#### 4. interactions

In [62]:
unique_perc(interactions)

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
0,1.0,0.04994,0.00004,0.00015,0.92371,0.00301,0.00035,0.99276


In [63]:
unique_vals(interactions, ["channel", "interaction_type", "page_or_product"], "interactions")

#### 5. support_tickets

In [64]:
unique_perc(support_tickets)

,ticket_id,customer_id,issue_category,priority,submission_date,resolution_date,resolution_status,resolution_time_hours,customer_satisfaction_score,notes
0,1.0,0.732,0.002667,0.001,0.382667,0.857333,0.001333,0.042,0.001667,0.124667


In [65]:
unique_vals(support_tickets, ["issue_category", "priority", "resolution_status"], "support_tickets")

#### 6. transactions

In [66]:
unique_perc(transactions)

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
0,1.0,0.142994,0.002353,0.000495,0.001393,0.857037,0.054529,0.000341,0.000217,0.000217


In [67]:
unique_vals(transactions, ["product_name", "product_category", "store_location", "payment_method"], "transactions")

### Rectify

In [68]:
def title_conv (df):
    for column in df.columns:
        if pd.api.types.is_string_dtype(df[column].dtype):
            df[column] = df[column].str.title()

In [69]:
title_conv(campaigns)
title_conv(customers)
title_conv(customer_reviews_complete)
title_conv(interactions)
title_conv(support_tickets)
title_conv(transactions)

## Duplication

### Discover

In [70]:
print(campaigns.duplicated().sum())
print(customers.duplicated().sum())
print(customer_reviews_complete.duplicated().sum())
print(interactions.duplicated().sum())
print(support_tickets.duplicated().sum())
print(transactions.duplicated().sum())

0
0
0
0
0
0


### Rectify

# Export

In [71]:
path_out = "../data/processed/"

campaigns.to_csv(path_out+"campaigns.csv")
customers.to_csv(path_out+"customers.csv")
customer_reviews_complete.to_csv(path_out+"customer_reviews_complete.csv")
interactions.to_csv(path_out+"interactions.csv")
support_tickets.to_csv(path_out+"support_tickets.csv")
transactions.to_csv(path_out+"transactions.csv")